# ID3算法
输入：训练数据集D，特征集A，阈值epsilon；
输出：决策树T
（1）若D中所有实例属于同一类Ck，则T为单结点树，并将类Ck作为该结点的类标记，返回T；

（2）若A=∅，则T为单结点树，并将D中实例最大的类Ck作为该结点的类标记，返回T；

（3）否则，计算A中各特征对D的信息增益，选择信息增益最大的特征Ag；

（4）如果Ag的信息增益小于阈值epsilon，则置T为单结点树，并将D中实例数最大的类Ck作为该结点的类标记，返回T；

（5）否则，对Ag的每一可能值ai，依Ag=ai将D分割为若干非空子集Di，将Di中实例数最大的类作为标记，构建子结点，由结点及其子结点构成树T，返回T；

（6）对第i个子结点，以Di为训练集，以A-{Ag}为特征集，递归地调用步（1）~步（5），得到子树Ti，返回Ti；

<img src='./ID3_tree_pics/formula1.png'></img>

<img src='./ID3_tree_pics/formula2.png'></img>

<img src='./ID3_tree_pics/formula3.png'></img>

In [13]:
import operator
import math


def create_data():
    dataSets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']

    # 返回数据集和每个维度的名称
    return dataSets, labels



class Node:
    def __init__(self, label=None, featureIndex=None,feature_name=None, 
                 parentNode=None, leaf=False,entropy=0):
        self.label = label
        self.leaf = leaf  # 是否为叶结点
        self.featureIndex = featureIndex
        self.feature_name = feature_name
        self.parentNode = parentNode
        self.childNodeList = {}  # 存的是子结点的取值，如feature_name是身高，可能存的是高，矮
        self.entropy=entropy
        self.sampleLists=[]
        self.prune_visited=False


    def predict(self,node,Xtest):
        if node.leaf:
            return node.label
        else:
            currentFeatIndex=node.featureIndex
            currentFeatVal=Xtest[currentFeatIndex]
            return self.predict(node.childNodeList[currentFeatVal],Xtest)



class dTree:
    def __init__(self, epsilon=0.1,alpha=0.3):
        self.epsilon = epsilon
        self.myTree = None
        self.completeDataSets=None
        self.completeAttrs=None
        self.leafNums=0
        self.alpha=alpha  #用于剪枝


    def treeGenerate(self,dataSets,labels):
        self.myTree=Node(parentNode='root')
        self.completeDataSets=dataSets.copy()
        self.completeAttrs=labels.copy()
        self.recursive(dataSets,labels,self.myTree)
        print("generate tree success!")


    def recursive(self, dataSets, labels, node=Node()):
        if self.isSameClass(dataSets):
            node.label = dataSets[0][-1]
            node.leaf = True
            node.entropy=self.calcEntropy(dataSets)
            node.sampleLists=dataSets
            self.leafNums+=1
            return
        if len(labels[:-1]) == 0:
            node.label = self.majorityClass(dataSets)
            node.leaf = True
            node.entropy = self.calcEntropy(dataSets)
            node.sampleLists = dataSets
            self.leafNums += 1
            return

        bestFeatIndex = self.chooseBestFeatureToSplit(dataSets, labels)
        node.featureIndex = bestFeatIndex
        node.feature_name = self.completeAttrs[bestFeatIndex]
        node.entropy=self.calcEntropy(dataSets)
        node.sampleLists=dataSets
        featList = [data[bestFeatIndex] for data in self.completeDataSets]
        uniqueFeatList = set(featList)
        for feat in uniqueFeatList:
            subNode = Node()
            node.childNodeList[feat]=subNode
            subNode.parentNode = node
            subDataSets = self.splitData(dataSets, bestFeatIndex, feat)
            if len(subDataSets) == 0:
                subNode.label = self.majorityClass(dataSets)
                subNode.leaf = True
                self.leafNums += 1
            else:
                subLabels = labels.copy()
                subLabels.remove(self.completeAttrs[bestFeatIndex])
                self.recursive(subDataSets, subLabels, subNode)



    def chooseBestFeatureToSplit(self, dataSets, labels):
        baseEntropy = self.calcEntropy(dataSets)
        m = len(dataSets)
        bestInfoGain = -math.inf
        bestFeatIndex = -1
        for i in range(len(labels) - 1):  # 遍历每个属性
            j=self.completeAttrs.index(labels[i])
            featList = [data[j] for data in dataSets]
            uniqueFeats = set(featList)  # 当前属性的取值
            newEntropy = 0
            for feat in uniqueFeats:  # 遍历当前属性的每个取值
                subDataSets = self.splitData(dataSets, j, feat)
                prob = len(subDataSets) / m
                newEntropy += prob * self.calcEntropy(subDataSets)
            currentInfoGain = baseEntropy - newEntropy
            if currentInfoGain >= bestInfoGain:
                bestInfoGain = currentInfoGain
                bestFeatIndex = j
        return bestFeatIndex


    def splitData(self, dataSets, featIndex, featVals):
        subDataSets = [data for data in dataSets if data[featIndex] == featVals]
        return subDataSets


    def calcEntropy(self, dataSets):
        informationEnt = 0
        m = len(dataSets)
        classCountDict = self.classCount(dataSets)
        for key in classCountDict.keys():
            prob = classCountDict[key] / m
            if prob == 0.0:
                Ent = 0.0
            else:
                Ent = prob * math.log(prob,2)
            informationEnt -= Ent
        return informationEnt

    def isSameClass(self, dataSets):
        C = dataSets[0][-1]  # 第一个样本的类
        for data in dataSets:
            if C != data[-1]:
                return False
        return True



    def majorityClass(self, dataSets):
        classCountDict = self.classCount(dataSets)
        sortedClassCount = sorted(classCountDict.items(), key=operator.itemgetter(1),reverse=True)
        return sortedClassCount[0][0]


    def classCount(self,dataSets):
        classCountDict = {}
        for data in dataSets:
            if data[-1] not in classCountDict.keys():
                classCountDict[data[-1]] = 1
            else:
                classCountDict[data[-1]] += 1
        return classCountDict

#后剪枝
    def postPruning(self):
        count=0
        leafParentNode = self.getTheLeafParentNode(self.myTree)  #找到一个子结点全是叶结点的结点，并且该结点的prune_visited=False
        while leafParentNode.parentNode!='root':  #找到根结点时停止
            beforeLoss = self.compute_recursive(self.myTree)+self.alpha*self.leafNums
            tmpLeafChildNode = leafParentNode.childNodeList.copy()  #将该结点的子结点保存起来
            leafParentNode.childNodeList = {}  #先假设将子结点剪了
            afterLoss = self.compute_recursive(self.myTree)+self.alpha*(self.leafNums-len(tmpLeafChildNode)+1)
            if afterLoss > beforeLoss:  # 如果剪了后损失值更大了，就不剪
                leafParentNode.childNodeList = tmpLeafChildNode
            else:
                leafParentNode.leaf=True
                leafParentNode.label=self.majorityClass(leafParentNode.sampleLists)
                self.leafNums=self.leafNums-len(tmpLeafChildNode)+1
                print("prune node: %s"%(leafParentNode.feature_name))
                count+=1
            leafParentNode = self.getTheLeafParentNode(self.myTree)  #寻找下一个子结点全是叶结点的结点
        print("prune finished,cut %d node!"%count)


    def compute_recursive(self,tree):
        loss=0
        if tree.childNodeList:  #如果当前结点有子结点
            for featVal,childNode in tree.childNodeList.items():
                loss+=self.compute_recursive(childNode)
        else:   #当前结点为叶结点
            loss+=len(tree.sampleLists)*tree.entropy
        return loss


    def getTheLeafParentNode(self,tree):
        isAllChildNodeLeaf=True
        if tree.childNodeList:  #如果当前结点有子结点，即不是叶结点
            for key,childNode in tree.childNodeList.items():#遍历当前结点的所有子结点
                if childNode.prune_visited:
                    continue
                if childNode.childNodeList:  #当前迭代下的子结点如果还有子结点，即不是叶节点的话
                    isAllChildNodeLeaf=False
                    return self.getTheLeafParentNode(childNode)  #递归地对非叶结点调用此方法
            if isAllChildNodeLeaf and not tree.prune_visited:   #如果当前结点所有子结点都是叶结点
                tree.prune_visited=True
                return tree



    def predict(self,Xtest):
        return self.myTree.predict(self.myTree,Xtest)


    def score(self,testDatas):
        #testDatas需要是至少有两个样本的list
        y=[data[-1] for data in testDatas]
        predictY=[]
        for Xtest in testDatas:
            predictY.append(self.predict(Xtest[:-1]))
        m=len(y)
        count=0
        for i in range(m):
            if y[i]==predictY[i]:
                count+=1
        accuracy=float(count/m)*100
        print(y)
        print(predictY)
        print("accuracy is "+str(accuracy)+"%")





In [14]:
firstDataSets,firstLabels=create_data()
firstMyTree=dTree()
firstMyTree.treeGenerate(firstDataSets,firstLabels)
firstMyTree.score(firstDataSets)
firstMyTree.postPruning()

generate tree success!
['否', '否', '是', '是', '否', '否', '否', '是', '是', '是', '是', '是', '是', '是', '否']
['否', '否', '是', '是', '否', '否', '否', '是', '是', '是', '是', '是', '是', '是', '否']
accuracy is 100.0%
prune finished,cut 0 node!


# 第一组数据生成的决策树

![image.png](./ID3_tree_pics/Figure_1.png)

In [15]:
secondDataSets=[
['青绿','蜷缩','浊响','清晰','凹陷','硬滑',1],
['乌黑','蜷缩','沉闷','清晰','凹陷','硬滑',1],
['乌黑','蜷缩','浊响','清晰','凹陷','硬滑',1],
['青绿','蜷缩','沉闷','清晰','凹陷','硬滑',1],
['浅白','蜷缩','浊响','清晰','凹陷','硬滑',1],
['青绿','稍蜷','浊响','清晰','稍凹','软粘',1],
['乌黑','稍蜷','浊响','稍糊','稍凹','软粘',1],
['乌黑','稍蜷','浊响','清晰','稍凹','硬滑',1],
['乌黑','稍蜷','沉闷','稍糊','稍凹','硬滑',0],
['青绿','硬挺','清脆','清晰','平坦','软粘',0],
['浅白','硬挺','清脆','模糊','平坦','硬滑',0],
['浅白','蜷缩','浊响','模糊','平坦','软粘',0],
['青绿','稍蜷','浊响','稍糊','凹陷','硬滑',0],
['浅白','稍蜷','沉闷','稍糊','凹陷','硬滑',0],
['乌黑','稍蜷','浊响','清晰','稍凹','软粘',0],
['浅白','蜷缩','浊响','模糊','平坦','硬滑',0],
['青绿','蜷缩','沉闷','稍糊','稍凹','硬滑',0]]

secondLabels=[u'色泽', u'根蒂', u'敲声', u'纹理', u'脐部', u'触感',u'类别']


secondMyTree=dTree()
secondMyTree.treeGenerate(secondDataSets,secondLabels)
secondMyTree.score(secondDataSets)
secondMyTree.postPruning()

generate tree success!
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
accuracy is 100.0%
prune finished,cut 0 node!


# 第二组数据生成的决策树

![image.png](./ID3_tree_pics/Figure_2.png)

In [16]:
thirdDataSets=[
['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 1],
['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 1],
['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 1],
['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 1],
['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', 1],
['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0],
['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', 0],
['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0],
['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', 0],
['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', 0]]

thirdLabels=[u'色泽', u'根蒂', u'敲声', u'纹理', u'脐部', u'触感',u'类别']

thirdTestDataSets=[
['青绿','蜷缩','沉闷','清晰','凹陷','硬滑',1],
['浅白','蜷缩','浊响','清晰','凹陷','硬滑',1],
['乌黑','稍蜷','浊响','清晰','稍凹','硬滑',1],
['乌黑','稍蜷','沉闷','稍糊','稍凹','硬滑',0],
['浅白','硬挺','清脆','模糊','平坦','硬滑',0],
['浅白','蜷缩','浊响','模糊','平坦','软粘',0],
['青绿','稍蜷','浊响','稍糊','凹陷','硬滑',0]]

thirdMyTree=dTree()
thirdMyTree.treeGenerate(thirdDataSets,thirdLabels)
thirdMyTree.score(thirdDataSets)
thirdMyTree.postPruning()

generate tree success!
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
accuracy is 100.0%
prune finished,cut 0 node!


# 第三组数据生成的决策树

![image.png](./ID3_tree_pics/Figure_3.png)

In [12]:
thirdMyTree.score(thirdTestDataSets)

[1, 1, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
accuracy is 85.71428571428571%


# 为何以信息增益作为划分训练数据集的特征，存在偏向于选择取值较多的特征的问题：
可以这样想假如这个特征取值特别多，每个样本就有一个取值，如果选择这个特征，那么每个分支都只有一条实例，也就是每个分支都属于同一个类，这个分支的熵就是0，这个特征的条件熵也就是0，那么基于这个特征的信息增益g(D,A)=H(D)-0=H(D)，就会是最大的

# 决策树的剪枝


决策树的剪枝分为预剪枝和后剪枝，预剪枝则是判断划分当前结点后验证集精确度是否提高（西瓜书），或者是损失函数值是否减少（统计学习方法），若是，则划分。而后剪枝是先生成好决策树，递归地试试删除结点能否提高验证集精确度或者减少损失函数值，若是，则剪枝。本模型只实现了postpruning（后剪枝）

# 对于连续属性的处理

给定样本集D和连续属性a，假定a在D上出现了n个不同的取值，将这些值从小到大进行排序，记为{a1,a2,...,an}，基于划分点t可将D分为子集Dt-,Dt+，Dt-包含a上取值不大于t的样本，而Dt+则包含那些在属性a上取值大于t的样本Gain(D,a)=max Gain(D,a,t)

In [17]:
def chooseBestFeatureToSplit(dataSet,contiualLabels):  #dataSet只包含连续属性
    baseEntropy=self.calcEntropy(labels)
    bestInfoGain=-math.inf
    bestT=-1
    for k in range(len(contiualLabels)):
        featList = [float(example[k]) for example in dataSets]
        sortedFeatList=featList.copy()
        sortedFeatList.sort()
        #求Ta
        for j in range(len(featList)-1):
            InfoGain = 0
            T=0.5*(sortedFeatList[j]+sortedFeatList[j+1])
            posDataSets,negDataSets=splitDataByT(dataSets,k,T)
            InfoGain=InfoGain+((len(posDataSets)/len(dataSet))*calcEntropy(posDataSets))+((len(negDataSets)/len(dataSet))*calcEntropy(negDataSets))
            Gain=baseEntropy-InfoGain
            if Gain > bestInfoGain:
                bestInfoGain = Gain
                bestFeature = k
                bestT=T
    return bestFeature,bestT


def splitDataByT(dataSets,featIndex,T):
    posDataSets=[data for data in dataSets if data[featIndex]>T]
    negDataSets=[data for data in dataSets if data[featIndex]<=T]
    return posDataSets,negDataSets
    


连续属性的处理见上面代码
需注意的是，与离散属性不同，若当前结点划分属性为连续属性，该属性还可作为其后代结点的划分属性。
例如在父结点上使用了“密度<=0.381”不会禁止在子结点上使用“密度<=0.294”